In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
#Started training at 5:10PM 1st April

#Dimensions of our flicker images is 256 X 256
img_width, img_height = 256, 256

#Declaration of parameters needed for training and validation
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3600 #1200 training samples for each class
nb_validation_samples = 1200 #400 training samples for each class
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#First convolution layer
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Second convolution layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Third convolution layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten the outputs of the convolution layer into a 1D contigious array
model.add(Flatten())

#Add a fully connected layer containing 128 neurons, use activation as relu and a dropout rate of 0.5
model.add(Dense(128, activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(rate=0.5))

#Add another fully connected layer containing 128 neurons, use activation as relu and a dropout rate of 0.5
model.add(Dense(128, activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(rate=0.5))

#Add the ouput layer containing 3 neurons, because we have 3 categories
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

#We will use the below code snippet for data augmentation on the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.4,
                                   zoom_range=0.4,
                                   vertical_flip=True,
                                   rotation_range=30,
                                   horizontal_flip=True)

#We won't augment the test data. We will just use ImageDataGenerator to rescale.
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size)


model.save_weights('cnn_from_scratch_weights.h5') 
model.save('cnn_from_scratch_model.h5') #Load using: model = load_model('cnn_from_scratch_weights.h5') from keras.models import load_model

Found 3600 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.
Epoch 1/50
225/225 [==============================] - 328s 1s/step - loss: 1.1454 - acc: 0.4378 - val_loss: 0.9657 - val_acc: 0.5775
Epoch 2/50
225/225 [==============================] - 336s 1s/step - loss: 1.0266 - acc: 0.5656 - val_loss: 0.7402 - val_acc: 0.6992
Epoch 3/50
225/225 [==============================] - 315s 1s/step - loss: 0.9132 - acc: 0.6731 - val_loss: 0.6665 - val_acc: 0.7433
Epoch 4/50
225/225 [==============================] - 305s 1s/step - loss: 0.8339 - acc: 0.7011 - val_loss: 0.9677 - val_acc: 0.6967
Epoch 5/50
225/225 [==============================] - 315s 1s/step - loss: 0.7745 - acc: 0.7364 - val_loss: 0.6850 - val_acc: 0.7817
Epoch 6/50
225/225 [==============================] - 330s 1s/step - loss: 0.7353 - acc: 0.7383 - val_loss: 0.6543 - val_acc: 0.7833
Epoch 7/50
225/225 [==============================] - 331s 1s/step - loss: 0.7250 - acc: 0.7450 - val_loss: 0.9312 - v

In [ ]:
#This function is used to plot/update the train and test loss after each epoch.
def plt_dynamic_loss(x, vy, ty, ax, colors=['b']):
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

#Get model history
history=model.history

#Plot the train and test loss vs number of epochs
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test score (Validation Loss):', score[0]) 
print('Test accuracy (Accuracy on Unseen Data):', score[1])

#Plot train vs test loss
fig,ax = plt.subplots(1,1)
ax.set_xlabel('Epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

#List of epoch numbers
x = list(range(1,nb_epoch+1))

#Display the loss
val_loss = history.history['val_loss'] #Validation Loss
loss = history.history['loss'] #Training Loss
plt_dynamic_loss(x, val_loss, loss, ax)